In [1]:
#import common librarie
from zipfile import ZipFile 
import os.path
from os import path
import pandas as pd
import numpy as np
import base64
import re
from itertools import groupby
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns



# Baseline Model

## Load Data

In [2]:
file_name = "poems_with_new_tags.zip"
poem_df = pd.read_csv('poems_with_new_tags.zip', compression='zip', header=0, quotechar='"')
poem_df.head()

,content,dynasty,author,tags,star,author_stars,title,tags_list,new_tags,new_first_tag,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
0,《吴都赋》云：“户藏烟浦，家具画船。”唯吴兴为然。春游之盛，西湖未能过也。己酉岁，予与萧时父...,宋代,姜夔,春游;怀人;,64,279,琵琶仙·《吴都赋》云：「户藏烟浦,"['春游', '怀人']",政治;游玩,政治,0,0,0,0,0,1,0,1
1,《廿一史弹词》第三段说秦汉开场词滚滚长江东逝水，浪花淘尽英雄。是非成败转头空。青山依旧在，几...,明代,杨慎,咏史;抒怀;人生;哲理,3244,131,临江仙·滚滚长江东逝水,"['咏史', '抒怀', '人生', '哲理']",写物;劝勉;悲苦;政治,写物,1,1,0,0,1,1,0,0
2,《水经》云：“彭蠡之口有石钟山焉。”郦元以为下临深潭，微风鼓浪，水石相搏，声如洪钟。是说也，...,宋代,苏轼,古文观止;纪游;写景;写山,306,4011,石钟山记,"['古文观止', '纪游', '写景', '写山']",写物;政治;游玩,写物,1,0,0,0,0,1,0,1
3,【序】辛亥之冬，予载雪诣石湖。止既月，授简索句，且征新声，作此两曲。石湖把玩不已，使工妓隶习...,宋代,姜夔,咏物;梅花,74,279,暗香疏影,"['咏物', '梅花']",劝勉,劝勉,0,1,0,0,0,0,0,0
4,〔一枝花〕 攀出墙朵朵花，折临路枝枝柳。花攀红蕊嫩，柳折翠条柔，浪子风流。凭着我折柳攀花手...,元代,关汉卿,;散曲;抒情;生活,140,124,【南吕】一枝花不伏老,"['散曲', '抒情', '生活']",写物;快乐;悲苦,写物,1,0,0,1,1,0,0,0


In [3]:
# Pre-processed content for bert embdedding
def processed_content_with_multi_sentences(poem_df):
    poem_contents = poem_df['content'].to_list()
    processed_poem_contents = []
    local_max_length = 0
    max_length = 30
    max_length_sentence = ""
    for poem in poem_contents:
        split_sentences = re.split('。|！|？',poem)
        prcessed_sentences = []
        for sentence in split_sentences:
            sentence = sentence.strip()
            sentence = sentence.replace("\u3000"," ")
            if "“" in sentence and "”" not in sentence:
                sentence = sentence + "”"
            elif "”" in sentence and "“" not in sentence:
                sentence = sentence[1:]
            elif len(re.findall(r'[\u4e00-\u9fff]+', sentence)) < 1:
                continue
            if len(sentence) > local_max_length:
                max_length_sentence = sentence
                local_max_length = max(local_max_length, len(sentence))
            sentence = "[CLS]" + sentence + "。"
            prcessed_sentences.append(sentence)

        processed_poem_string = "[SEP]".join(prcessed_sentences)
        processed_poem_contents.append(processed_poem_string)

    print(local_max_length)  
    print(max_length_sentence)

def processed_content_with_one_poems(poem_df):
    poem_contents = poem_df['content'].to_list()
    processed_poem_contents = []
    max_length = 512
    max_length_sentence = ""
    for poem in poem_contents:

SyntaxError: unexpected EOF while parsing (<ipython-input-3-b2557c350962>, line 37)

In [4]:
poem_df["processed_content"] = processed_poem_contents

## Split Train and Test Data

In [5]:
poems_info = poem_df[['content','dynasty','author','title']]
tags = poem_df.drop(['content','dynasty','author','title','star','author_stars'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(poems_info, tags, test_size=0.33)
y_train_final = y_train.drop(['tags','tags_list','new_tags','new_first_tag'], axis=1)
y_test_final = y_test.drop(['tags','tags_list','new_tags','new_first_tag'], axis=1)

In [6]:
print(len(X_train)==len(y_train))

True


In [7]:
X_train.head()

,content,dynasty,author,title
3389,汉昭帝逝世，昌邑王刘贺被废黜，汉宣帝刘询刚刚登上皇位。路温舒呈上奏书，奏书说： 昭帝崩，昌...,两汉,路温舒,尚德缓刑书
396,买陂塘、旋栽杨柳，依稀淮岸江浦。东皋嘉雨新痕涨，沙觜鹭来鸥聚。堪爱处最好是、一川夜月光流渚。...,宋代,晁补之,摸鱼儿·东皋寓居
4665,胜神鳌，夯风涛，脊梁上轻负着蓬莱岛。万里夕阳锦背高，翻身犹恨东洋小，太公怎钓？,元代,王和卿,拨不断·大鱼
4632,老农家贫在山住，耕种山田三四亩。苗疏税多不得食，输入官仓化为土。岁暮锄犁傍空室，呼儿登山收橡...,唐代,张籍,野老歌山农词
4250,离人无语月无声，明月有光人有情。别后相思人似月，云间水上到层城。,唐代,李冶,明月夜留别


In [8]:
X_train.shape

(4091, 4)

In [9]:
y_train_final.head()

,写物,劝勉,家庭,快乐,悲苦,政治,朋友,游玩
3389,1,0,0,0,0,1,0,0
396,0,1,0,0,0,1,0,1
4665,0,1,0,0,1,1,0,0
4632,1,0,0,1,1,1,0,1
4250,1,0,0,0,0,0,1,0


In [10]:
y_train_final.shape

(4091, 8)

## Bert Tokenizing

In [14]:
from transformers import BertTokenizer, RobertaTokenizer, DistilBertTokenizer,TFBertModel


In [15]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

In [37]:
def convert_poem_to_feature(review):
    bert_input = bert_tokenizer.encode_plus(review, 
                                 add_special_tokens = True, # add [CLS], [SEP]
                                 max_length = 512,
                                 pad_to_max_length = True, # add [PAD] tokens
                                 return_attention_mask = True # add attention mask to not focus on pad tokens
                                )
    return bert_input

# map to the expected input to TFBertForSequenceClassification, see here 
def map_poem_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
      "input_ids": input_ids,
      "token_type_ids": token_type_ids,
      "attention_mask": attention_masks,
  }, label

def encode_poems(poems, labels,limit=-1):
    embedding_weight = []
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    label_list = []
    # prepare list, so that we can build up final TensorFlow dataset from slices.
    if (limit > 0):
        ds = ds.take(limit)

    for poem, label in zip(poems['content'],labels):
        bert_input = convert_poem_to_feature(poem)
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append(label.tolist())
    return (input_ids_list, token_type_ids_list, label_list)    
#     return tf.data.Dataset.from_tensor_slices((input_ids_list, 
#                                                attention_mask_list, 
#                                                token_type_ids_list, label_list)).map(map_poem_to_dict)
                                                                                                                         
                                                                                                                         
                                                                                                                         
                                                                                                                        

In [38]:
#test
small_example_train = X_train[:10]
small_example_label = y_train_final[:10].to_numpy()
result= encode_poems(small_example_train,small_example_label)
print(result[2])

[[1, 0, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1], [0, 1, 0, 0, 1, 1, 0, 0], [1, 0, 0, 1, 1, 1, 0, 1], [1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 1, 1, 0, 1, 0], [1, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 1], [1, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0, 0]]


In [32]:
# train dataset
batch_size = 64
ds_train_encoded, ds_train_labels = encode_poems(X_train,y_train_final.to_numpy())
# test dataset
ds_test_encoded, ds_test_labels = encode_poems(X_test, y_test_final.to_numpy())

In [33]:
len(bert_tokenizer.vocab)

21128

## Other Models

In [34]:
from numpy import mean
from numpy import std
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from transformers import TFBertForSequenceClassification

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [30]:
# # recommended learning rate for Adam 5e-5, 3e-5, 2e-5
# learning_rate = 2e-5
# # we will do just 1 epoch for illustration, though multiple epochs might be better as long as we will not overfit the model
# number_of_epochs = 8

# # model initialization
# model = TFBertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=8)

# # optimizer Adam recommended
# optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate,epsilon=1e-08, clipnorm=1)

# # we do not have one-hot vectors, we can use sparce categorical cross entropy and accuracy
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
# model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
# # fit model
# bert_history = model.fit(ds_train_encoded, epochs=number_of_epochs)
# # evaluate test set
# model.evaluate(ds_test_encoded)

In [ ]:
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(X_train_tokenized)

# X_train = tokenizer.texts_to_sequences(X_train_tokenized)
# X_test = tokenizer.texts_to_sequences(X_test_tokenized)

# vocab_size = len(tokenizer.word_index) + 1

# maxlen = 200

# X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
# X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


In [35]:
from numpy import array
from numpy import asarray
from numpy import zeros

# embeddings_dictionary = dict()

# glove_file = open('/content/drive/My Drive/Colab Datasets/glove.6B.100d.txt', encoding="utf8")

# for line in glove_file:
#     records = line.split()
#     word = records[0]
#     vector_dimensions = asarray(records[1:], dtype='float32')
#     embeddings_dictionary[word] = vector_dimensions
# glove_file.close()

# embedding_matrix = zeros((vocab_size, 100))
# for word, index in tokenizer.word_index.items():
#     embedding_vector = embeddings_dictionary.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[index] = embedding_vector

In [12]:
# bert_layer = TFBertModel.from_pretrained('bert-base-chinese')

Some layers from the model checkpoint at bert-base-chinese were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-chinese.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [30]:
# maxlen = 200
# tokenizer = Tokenizer(num_words=5000)
# tokenizer.fit_on_texts(X_train_tokenized)
# X_train = tokenizer.texts_to_sequences(X_train_tokenized)
# X_test = tokenizer.texts_to_sequences(X_test_tokenized)
# X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
# X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [36]:
deep_inputs = Input(shape=(513,))
embedding_layer = Embedding(21128, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(6, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [81]:
# model = Sequential()
# model.add(Embedding(21128, 64))
# model.add(Flatten())
# model.add(Dense(1, activation='sigmoid'))
# # compile the model
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
# # summarize the model
# print(model.summary())
# # fit the model
# model.fit(ds_train_encoded, epochs=50, verbose=0)
# # evaluate the model
# loss, accuracy = model.evaluate(ds_test_encoded, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

In [ ]:
# deep_inputs = Input(shape=(maxlen,))
# embedding_layer = Embedding(34345, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
# LSTM_Layer_1 = LSTM(128)(embedding_layer)
# dense_layer_1 = Dense(6, activation='sigmoid')(LSTM_Layer_1)
# model = Model(inputs=deep_inputs, outputs=dense_layer_1)

# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
